<a href="https://colab.research.google.com/github/CanopySimulations/canopy-python-examples/blob/master/loading_all_vector_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Upgrade Runtime
This cell ensures the runtime supports `asyncio` async/await, and is needed on Google Colab. If the runtime is upgraded, you will be prompted to restart it, which you should do before continuing execution.

In [ ]:
!pip install ipython ipykernel --upgrade

# Set Up Environment

### Import required libraries

In [ ]:
!pip install -q canopy

In [ ]:
import canopy
import logging
import nest_asyncio

logging.basicConfig(level=logging.INFO)
nest_asyncio.apply()

### Authenticate

In [ ]:
authentication_data = canopy.prompt_for_authentication()
session = canopy.Session(authentication_data)

# Set Up Example

### Run a simulation using default configs

In [ ]:
default_car_name = 'Canopy F1 Car 2019'
default_weather_name = '25 deg, dry'
   
logging.info('Loading configs...')

default_car = await canopy.load_default_config(session, 'car', default_car_name)
default_weather = await canopy.load_default_config(session, 'weather', default_weather_name)

logging.info('Creating study...')

study_id = await canopy.create_study(
    session,
    'apexSim',
    'Python API Test Study',
    [
        default_car,
        default_weather,
    ])

logging.info('Waiting for study to complete...')

wait_result = await canopy.wait_for_study(
    session,
    study_id,
    timeout_seconds=60)

logging.info('Sim Version: {}'.format(wait_result.document.sim_version))    
logging.info('Succeded Simulations: {}/{}'.format(
    wait_result.succeeded_simulation_count, 
    wait_result.simulation_count))

INFO:root:Loading configs...
INFO:root:Creating study...
INFO:root:Waiting for study to complete...
INFO:root:Sim Version: 1.3371
INFO:root:Succeded Simulations: 1/1


# Example: Load all channel data
_Note:_ For performance reasons it is usually recommended to load only the channels you need. However loading all channels is possible by first fetching the channel list from the vector metadata.

In [ ]:
job_metadata = await canopy.load_study_job(
    session,
    study_id,
    'ApexSim',
    include_vector_metadata=True)

job_metadata.vector_metadata

INFO:canopy.load_study_job:Loading job index 0


,units,description,NPtsInChannel,xDomainName
name,,,,
xCar,m,x position of the car centre of gravity in the...,42,vCar
yCar,m,y position of the car centre of gravity in the...,42,vCar
zCar,m,z position of the car centre of gravity in the...,42,vCar
aRoll,rad,Roll angle of the car in the world coordinate ...,42,vCar
aPitch,rad,Pitch angle of the car in the world coordinate...,42,vCar
...,...,...,...,...
aSlipTyreR,rad,Average slip angle of the rear tyres.,42,vCar
zTrackFL,m,Track height at the front left contact patch.,42,vCar
zTrackFR,m,Track height at the front right contact patch.,42,vCar


The vector_metadata is a DataFrame indexed by the channel name, so we can extract the list of channels by asking for the index column.


In [ ]:
all_channels = job_metadata.vector_metadata.index
all_channels

Index(['xCar', 'yCar', 'zCar', 'aRoll', 'aPitch', 'aYaw', 'vCar', 'dxCar',
       'dyCar', 'dzCar',
       ...
       'nWheelRR', 'dnWheelRR', 'aSteerKinematicRR', 'aSteerTotalRR',
       'aSlipTyreF', 'aSlipTyreR', 'zTrackFL', 'zTrackFR', 'zTrackRL',
       'zTrackRR'],
      dtype='object', name='name', length=398)

We can now pass that list back into the `load_study_job` function to load all the channel data.

In [ ]:
job_data = await canopy.load_study_job(
    session,
    study_id,
    'ApexSim',
    channel_names=all_channels)

logging.info('Loaded {} channels'.format(len(job_data.vector_data.columns)))

INFO:canopy.load_study_job:Loading job index 0
INFO:root:Loaded 398 channels


In [ ]:
job_data.vector_data

,xCar,yCar,zCar,aRoll,aPitch,aYaw,vCar,dxCar,dyCar,dzCar,nRoll,nPitch,nYaw,gLong,gLat,gVert,dnRoll,dnPitch,dnYaw,aSlipCar,aSteerWheel,aSteerL,aSteerR,gLongAccelerometer,gLatAccelerometer,gVertAccelerometer,xRackL,xRackR,d2xRackL,d2xRackR,FRackL,FRackR,hRideF,hRideR,aInGGPlane,rLatBalF,cUnderOverSteer,PEnergyKinetic,hRollCentreF,hRollCentreR,...,MyRollingResistanceRL,nWheelRL,dnWheelRL,aSteerKinematicRL,aSteerTotalRL,aSlipTyreRR,rSlipTyreRR,aCamberTyreRR,rTyreLoadedRR,xTyreSquashRR,rTyreRollingRadiusRR,xBumpRR,kHeaveTyreRR,PTyreWearRR,PTyreWearLatRR,PTyreWearLongRR,rGripTyreWearRR,rGripTyreRR,rGripLongTyreRR,rGripTemperatureRR,FxTyreRR,FyTyreRR,FzTyreRR,MxTyreRR,MyTyreRR,MzTyreRR,FxyTyreRR,dFxTyre_drSlipRR,dFyTyre_daSlipRR,MyRollingResistanceRR,nWheelRR,dnWheelRR,aSteerKinematicRR,aSteerTotalRR,aSlipTyreF,aSlipTyreR,zTrackFL,zTrackFR,zTrackRL,zTrackRR
0,0.0,0.0,-0.329205,-0.013379,-0.028217,0.0,9.894447,9.898388,1.421942,0.0,0.0,0.0,1.114998,-3.336731,10.781949,0.238442,0.016619,0.035064,4.213335e-08,0.143091,4.861622,0.419765,0.405655,-3.613499,10.913137,-9.566776,-0.038131,0.038536,1.776698e-09,4.046354e-10,-5560.691895,1518.888794,0.023581,0.125147,1.870922,0.331080,0.003166,-2.455135e+04,0.051559,0.031666,...,1.373228,31.928009,-5.854056,0.003029,0.003029,-0.039396,-0.003409,-0.023210,0.331405,0.003595,0.335022,0.053970,237013.187500,298.173035,295.878662,2.294368,1.0,1.0,1.0,1.0,-74.663605,833.077454,-879.377502,-154.336761,0.0,-43.457191,836.416565,23165.451172,17517.943359,0.324411,26.909277,-14.073802,-0.003287,-0.003287,-0.066421,-0.039138,0.0,0.0,0.0,0.0
1,0.0,0.0,-0.328914,-0.014794,-0.028288,0.0,11.929655,11.934430,1.252748,0.0,0.0,0.0,1.170574,-3.914475,13.708759,0.313599,0.020255,0.038745,-6.789500e-09,0.105030,4.685647,0.401126,0.391320,-4.191942,13.853827,-9.491403,-0.036552,0.037092,-5.443151e-07,1.281378e-07,-7353.789551,1946.950317,0.023155,0.124977,1.848940,0.408461,0.012537,-3.473087e+04,0.053728,0.032957,...,1.718715,38.090595,-6.580324,0.003018,0.003018,-0.055027,-0.003862,-0.024572,0.331819,0.003181,0.335032,0.054476,236266.328125,582.237671,579.205627,3.032036,1.0,1.0,1.0,1.0,-71.380508,957.091187,-784.023254,-145.515793,0.0,-45.217838,959.749329,20589.001953,12053.225586,0.351637,32.825977,-16.797375,-0.003302,-0.003302,-0.111041,-0.054091,0.0,0.0,0.0,0.0
2,0.0,0.0,-0.328387,-0.013980,-0.028380,0.0,13.964085,13.969710,0.920436,0.0,0.0,0.0,1.074178,-3.809408,14.815870,0.315293,0.016118,0.032732,1.547541e-08,0.066208,3.787333,0.317786,0.320081,-4.087776,14.952957,-9.489799,-0.029318,0.029986,-2.760214e-07,3.060544e-08,-8209.657227,1528.912231,0.022443,0.124597,1.822462,0.474335,0.011882,-3.956514e+04,0.054762,0.032078,...,1.970794,43.853542,-6.769568,0.003023,0.003023,-0.058975,-0.008937,-0.023813,0.331352,0.003648,0.335045,0.053935,237130.140625,903.479553,881.823181,21.656395,1.0,1.0,1.0,1.0,-185.932983,1147.309814,-897.596191,-157.842285,0.0,-76.905151,1162.278320,23359.474609,12744.864258,0.478913,38.898113,-15.979709,-0.003288,-0.003288,-0.108902,-0.058655,0.0,0.0,0.0,0.0
3,0.0,0.0,-0.327890,-0.013506,-0.028425,0.0,15.981847,15.988306,0.611621,0.0,0.0,0.0,0.984534,-3.803377,15.614357,0.319046,0.013080,0.027540,3.397721e-08,0.038631,3.178540,0.261697,0.271098,-4.082190,15.746790,-9.486097,-0.024393,0.025163,5.861522e-10,-7.067770e-12,-9008.933594,1309.382690,0.021856,0.124173,1.809726,0.513576,0.012478,-4.521255e+04,0.055278,0.031509,...,2.240149,49.577099,-7.116436,0.003023,0.003023,-0.061103,-0.012609,-0.023383,0.330980,0.004020,0.335061,0.053516,237626.703125,1241.639282,1187.419312,54.219978,1.0,1.0,1.0,1.0,-285.801483,1291.582764,-989.306335,-165.878540,0.0,-103.000931,1322.825928,25427.931641,13415.337891,0.610711,44.904881,-15.595737,-0.003277,-0.003277,-0.111657,-0.061345,0.0,0.0,0.0,0.0
4,0.0,0.0,-0.327377,-0.013286,-0.028450,0.0,17.989611,17.996895,0.334363,0.0,0.0,0.0,0.910572,-3.881589,16.316700,0.327272,0.011007,0.023579,-3.773585e-09,0.018961,2.748400,0.222126,0.236159,-4.160650,16.4469

Once these are loaded we can request individual channels in specific units as per usual.

In [ ]:
job_data.vector_as('hRideF', 'mm')

0     23.581018
1     23.154509
2     22.443340
3     21.855806
4     21.292604
5     20.719929
6     20.141708
7     19.575975
8     18.988611
9     18.526531
10    17.841379
11    17.124043
12    16.378126
13    15.615377
14    14.847624
15    14.077945
16    13.396502
17    12.594221
18    11.775552
19    10.903538
20    10.008116
21     9.089180
22     8.210709
23     7.243148
24     6.249192
25     5.230810
26     4.189162
27     3.125695
28     2.087222
29     0.978754
30    -0.153785
31    -1.312594
32    -2.494211
33    -3.684214
34    -4.926078
35    -6.198545
36    -7.492600
37    -8.785694
38    -9.897973
39   -10.383286
40   -10.745470
41   -11.093386
Name: hRideF, dtype: float32